In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

$M$ - is the number of states: 2 - fair (F) and biased (B) dice

$K$ - is the number of possible emissions: 6 - for every dice side

$L$ - is length. Let it be 250

In [2]:
L = 250

- Set transition ($M \times M$) and emission ($M \times K$) matrices and beginning distribution
(vector of length $M$).

In [3]:
#transition matrix: F->F, F->B, B->F, B->B
T = np.array([[.95, .05], [.1, .9]])
#emission matrix: 1/6 * 6 for F and .1 * 5, .5 for B
Em = np.array([[1/6 for i in range(6)], [.1, .1, .1, .1, .1, .5]])

beginning distribution $p_0$ - (P(throwing F first, P(throwing B first)). 
Distribution is stationar, so $p_0\cdot T= p_0$

In [4]:
#beginning distribution
p0 = (2/3, 1/3)

- HMM random generator: generate a sequence x of length L given the model parameters.

In [5]:
#helps us to choose one value from several, if we have the array of their probabilities
'''
1. sum of all probabilites is 1
2. choice is a random number from [0,1)
3. we substract probabilities from it till it is in [0,1)
4. the step, when it becomes negative, defines our choice
'''

def choose(probs):
    flag = np.random.random()
    i = 0
    for prob in probs:
        flag -= prob
        if flag < 0:
            return i
        i += 1

Now we have to create hidden and obsrved sequences

In [6]:
'''
Hidden sequence is responsible for dice choices
The first state is chosen according to beginning distribation. 
The next states are  - according to transition probabilities.
'''
def hidden_seq(start_dice, trans, length):
    seq = np.zeros(length, dtype=int)
    seq[0] = choose(start_dice)
    for i in range(1, length):
        dice = trans[seq[i-1]]
        seq[i] = choose(dice)
    return seq

In [7]:
'''
observed sequence (= sequence x) is responsible for dice faces
it is based on hidden sequence and emission probabilities
'''
def observed_seq(hidden, emission):
    length = len(hidden)
    seq = np.zeros(length, dtype=int)
    for i in range(length):
        face = emission[hidden[i]]
        seq[i] = choose(face)
    return seq

- Use Viterbi algorithm to find the most probable path π given x and HMM parameters.

<img src='1.png'></img>

In [19]:
#most likely dice choices, knowing dice faces
'''
x - observed sequence
a - transition matrix
e - emission matrix
'''
def viterbi(start_dice, x, a, e):
    L = len(x)
    V = np.zeros((2, L))
    ptr = np.zeros((L, 2))
    #initialisation
    '''
    I changed it comparing to what we had in tutorial - we know beginning distribution
    '''
    V[ :, 0] = start_dice * e[ :, x[0]]
    #recursion
    for i in range(1, L):
        for l in range(2):
            V[l, i] = e[l, x[i]] * np.max(V[ :, i - 1] * a[ :, l])
            ptr[i, l] = np.argmax(V[ :, i - 1] * a[ :, l])
    
    seq = np.zeros(L, dtype=int)
    #termination
    seq[-1] = np.argmax(V[ :, -1] * a[ :, -1])
    #traceback
    for i in range(L - 1, 0, -1):
        seq[i - 1] = ptr[i, seq[i]]
    return seq

In [22]:
viterbi(p0, observed_seq(hidden_seq(p0, T, L), Em), T, Em)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1])